# Not your grandmother’s toolbox– the Robotics Toolbox reinvented for python
### Peter Corke and Jesse Haviland

This is the code for the examples in the [paper published at ICRA2021](https://ieeexplore.ieee.org/document/9561366).


## Configuring CoLab

To make this run nicely in CoLab we have to install a bunch of packages.  This will be slow and at the end you will need to restart the kernel.  Push the `RESTART KERNEL` button that appears or from the menu above `Runtime/RestartRuntime`

In [ ]:
!pip install numpy --upgrade
!pip install roboticstoolbox-python
!pip install -U ipython
!pip install sympy
!pip install spatialgeometry[collision]

Now, we can import some of the packages we need throughout, NumPy and Matplotlib, and configure the CoLab environment a bit.

In [ ]:
from math import pi
import numpy as np
import matplotlib.pyplot as plt
# display result of assignments
%config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'
# make NumPy display a bit nicer
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:10.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})
# make cells nice and wide
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))
# %matplotlib widget

# III.SPATIAL MATHEMATICS

The spatial math package underpins the Robotics and Machine Vision Toolboxes. It supports mathematical objects such as SO(2), SE(2), SO(3) and SE(3) matrices, quaternions and unit quaternions as well as twists in 2D and 3D.

We start by importing the package

In [ ]:
from spatialmath.base import *

In robotics we most often need to represent pose in 3D space, and a common way to do this is with a matrix belonging to the group SE(3).

We can create an SE(3) transformation by composing a number of simple transformations such as translations and rotations.  The toolbox has a bunch of functions that create and manipulate:

* 2D NumPy arrays for SO(2), SE(2), SO(3) and SE(3), so(2), se(2), so(3), se(3) matrices
* 1D NumPy arrays for quaternions, unit-quaternions and Twists

If you are familiar with the venerable Robotics Toolbox for MATLAB, these functions have the same names and functionality.

In [ ]:
T = transl(0.5, 0.0, 0.0)

has created an SE(3) matrix that represents a pure translation of 0.5 in the x-direction.

In [ ]:
T = trotx(-90, 'deg')

has created an SE(3) matrix that represents a pure rotation of -90° about the y-axis

In [ ]:
rpy2tr(0.1, 0.2, 0.3, order='xyz')

has created an SE(3) matrix that represents a rotation by the roll-pitch-yaw angles respectively 0.1, 0.2 and 0.3 radians.  They are composed in the order:
 * yaw about the z-axis, 
 * pitch about the y-axis, then 
 * roll about the x-axis 

This is the `order` given as `"zyx"` which is a common convention for mobile robots and drones.

We can compose these SE(3) matrices by matrix multiplication (the NumPy `@` operator)

In [ ]:
T = transl(0.5, 0.0, 0.0) @ rpy2tr(0.1, 0.2, 0.3, order='xyz') @ trotx(-90, 'deg')

which has created a 4x4 NumPy array with an SE(3) matrix value.  There are lots of functions in `spatialmath.base` to convert between different representations such roll-pitch-yaw angles, Euler angles, angle-axis, unit quaternions and so on.  There are also functions to plot and animate frames and even turn those into mp4 files. If you're a fan of twists and product of exponential representations there are also functions to compute skew-symmetric matrices, logarithms and exponentials.

Alternatively, and definitely encouraged, we can use the object-oriented interface

In [ ]:
from spatialmath import *

and we can rewrite the above example as

In [ ]:
T = SE3(0.5, 0.0, 0.0) * SE3.RPY([0.1, 0.2, 0.3], order='xyz') * SE3.Rx(-90, unit='deg')

In most environments, we see the matrix printed with color coding: red for rotation submatrix, blue for translation, grey for constant values.

The matrices are encapsulated in objects, we can compose them using the regular `*` operator.  

In [ ]:
T.eul()

In [ ]:
T.R

In [ ]:
T.plot(color='red', label='2')

In [ ]:
T.inv()

We can create unit quaternions using the same methods as we did for `SE3` objects, and compose them using the `*` operator

In [ ]:
q1 = UnitQuaternion.Rx(0.3)


In [ ]:
q2 = UnitQuaternion.AngVec(0.3, [1, 0, 0])

In [ ]:
q3 = q1 * q2

The inverse is easy

In [ ]:
q3.inv()

and we can convert back to an SO(3) rotation matrix

In [ ]:
q3.SO3()

All the objects just introduced have three superpowers.  Firstly, they can each hold multiple values, just like a Python list.  This is really useful for a collection of frames along a trajectory, or for every link on a robot arm

In [ ]:
T_many = SE3.Rx(np.linspace(0, pi/2, num=100))


In [ ]:
len(T_many)

So we can slice them

In [ ]:
T_many[20]

or use them as iterators in `for` loop or comprehensions

In [ ]:
for T in T_many:
    pass  # do a thing

The second super power is broadcasting.

In [ ]:
T = SE3.Ry(60, unit="deg")
T2 = T_many * T
len(T2)

which has created a new `SE3` object where each value is `T_many[k] * T`.

The third superpower is all these objects can transform a vector, or a bunch of vectors

In [ ]:
T * [1, 2, 3]

and the class has handled all the hassle of converting the vector to homogeneous form first.

This also works for a collection of vectors, always arranged columnwise

In [ ]:
T * np.array([[1,2,3], [4,5,6]]).T

and combining all the superpowers we can write

In [ ]:
T_many * [1, 2, 3]

which has applied each element of `T_many` to transform the vector `[1,2,3]`, resulting in a 3x100 array, one column per transformed vector.

# IV. ROBOTICS TOOLBOX

## A. Robot models

If you like Denavit-Hartenberg notation (standard or modified) then it is easy to create a robot model, it is simply a list of objects that represent a link-joint pair

In [ ]:
from roboticstoolbox import *
# robot length values (metres)
d1 = 0.352
a1 = 0.070
a2 = 0.360
d4 = 0.380
d6 = 0.065;

In [ ]:
robot = DHRobot([
  RevoluteDH(d=d1, a=a1, alpha=-pi/2), 
  RevoluteDH(a=a2), 
  RevoluteDH(alpha=pi/2),
  ], name="my IRB140")

There are also a lot of prebuilt models shipping with the toolbox

In [ ]:
models.list()

There are currently over 50 robot models, and they fall into three types:
1. `DH`, based on standard or modifed Denavit-Hartenberg (DH) notation.  This is what you will find in most of the standard robotics texts
2. `ETS`, this is a new way to think about robot kinematics that avoids the (IMO) complexity of DH notation, and was described in [this paper](https://ieeexplore.ieee.org/document/4252158)
3. [`URDF`](http://wiki.ros.org/urdf/Tutorials/Building%20a%20Visual%20Robot%20Model%20with%20URDF%20from%20Scratch), which is an industry standard way to represent kinematics, dynamics and 3D structure in XML format

We will load a standard Denavit-Hartenberg model of Peter's favourite robot

In [ ]:
puma = models.DH.Puma560()

The `__repr__` method for these objects pretty prints the parameters as a table.  The second table is a list of named joint configurations, for example the "nominal" configuration is

In [ ]:
puma.qn

You can easily add your own, which modifies that particular robot instance

In [ ]:
puma.addconfiguration("foo", [1,2,3,4,5,6])
puma.foo

The most fundamental operation on a robot is to determine its forward kinematics, that is, the pose of its end effector given the joint configuration.  All types of robot object have an `fkine` method to compute this

In [ ]:
T = puma.fkine([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

or, if you prefer, in a more compact form

In [ ]:
T.printline()

The next most fundamental operation on a robot is to determine its inverse kinematics, that is, given the pose of its end effector, what are the required joint angles.  In general there are multiple solutions, and the Toolbox provides a numerical solution to this problem

In [ ]:
sol = puma.ikine_LM(T)
sol

and the return value indicates success and provides the required joint angles.

Because this is a numerical solution, it requires a reasonable initial estimate of the joint configuration.  Here it started at zero and has managed to converge, but if it does not converge then a better initial estimate is required, provided using the `q0=` argument.

Some robots, for example the Puma560, have an analytical solution available as well, and its options allow for control over which, of the multiple, solutions is returned

In [ ]:
puma.ikine_a(T)

We can also plot the robot, but this seems to be not working in Jupyter :)

In [ ]:
puma.plot(sol.q, backend="pyplot");

In [ ]:
puma.ikine_a(T, config="lun")

As mentioned earlier, the ETS notation is a different way to think about robot kinematics.

We start by defining a set of dimensions for the robot, this example mimics the one in the [paper](https://ieeexplore.ieee.org/document/4252158)

In [ ]:
from roboticstoolbox import ETS as ET
# Puma dimensions (m), see RVC2 Fig. 7.4 for details
l1 = 0.672
l2 = -0.2337
l3 = 0.4318
l4 = 0.0203
l5 = 0.0837
l6 = 0.4318;

and now we can write an expression, in terms of translations and rotations, along or about the x-, y- and z-axes.  These can be by a constant amount or by a joint variable (indicated by no passed argument to the method)

In [ ]:
e = ET.tz(l1) * ET.rz() * ET.ty(l2) * ET.ry() \
    * ET.tz(l3) * ET.tx(l4) * ET.ty(l5) * ET.ry() \
    * ET.tz(l6) * ET.rz() * ET.ry() * ET.rz()

The result is an `ETS` object which is like a Python list, it can sliced and iterated.

We pass this to the `ERobot` constructor (E for ETS robot) and it chops the ETS into chunks, each one ending with a joint variable

In [ ]:
robot = ERobot(e)

The `@` symbol indicates the end of a branch (we can have robots with multiple branches) or an end-effector.

Finally, we will load a model of the Franka-Emika Panda from a URDF file that is included with the Toolbox

In [ ]:
panda = models.URDF.Panda()

The table at the top has tags:
* `geometry` the model includes models of the 3D geometry of the robot, as STL or Collada files.
* `collision` the model includes collision geometry, a set of simple geometric primitives, such as cylinders and spheres, for each link that can be used for collision checking.

The `@` symbol again indicates the end of a branch (we can have robots with multiple branches) or an end-effector.  Links shown in blue have a constant transformation with respect to their parent link.


## B. Trajectories

In [ ]:
traj = jtraj(puma.qz, puma.qr, 100)
plt.plot(traj.q);

In [ ]:
t = np.arange(0, 2, 0.010)
T0 = SE3(0.6, -0.5, 0.3)

In [ ]:
T1 = SE3(0.4, 0.5, 0.2)

In [ ]:
Ts = ctraj(T0, T1, t)
len(Ts)

In [ ]:
sol = puma.ikine_LM(Ts)
sol.q.shape

## C. Symbolic manipulation

In [ ]:
import spatialmath.base.symbolic as sym

We first define some symbols, these are Python objects.  We can give them unicode names so that they display as Greek letters.  Now we can display the mapping from roll-pitch-yaw angles to an SO(3) matrix.

In [ ]:
phi, theta, psi = sym.symbol('φ, ϴ, ψ')
rpy2r(phi, theta, psi)

Next, we can define a vector of symbolic joint variables, and use that to compute the forward kinematics -- symbolically.

In [ ]:
q = sym.symbol("q_:6") # q = (q_1, q_2, ... q_5)
T = puma.fkine(q);

This expression is huge, remove the semicolon on the last line above, and rerun the cell to check.

Partly this is because $\alpha_j$ values, which are generally some integer multiple of $\pi/2$, lead to sin and cos values that are not quite zero or one.

Instead we can load a new model of the robot where those parameters have symbolic values, not numeric, leading to crisp zero or one trig results.

In [ ]:
puma = models.DH.Puma560(symbolic=True)
T = puma.fkine(q)

The result is much more compact.  Here, we will display just the translational part of this.  The position of the end effector as a function of the first three joint angles.  The last three joint angles don't enter because the robot has a spherical wrist and the tool in this example has zero length.

In [ ]:
T.t[0]

## D. Differential kinematics

We will reload the numerical robot model

In [ ]:
puma = models.DH.Puma560(symbolic=False)

Now we will compute the Jacobian matrix that relates joint velocity to spatial velocity of the end effector, in the world-coordinate frame

In [ ]:
J = puma.jacob0(puma.qr)

This particular configuration is singular, and the Jacobian is not full rank

In [ ]:
np.linalg.matrix_rank(J)

Analyzing dependencies amongst the columns of the Jacobians we find that

In [ ]:
jsingu(J)

We can just as easily compute the Jacobian in the end effector coordinate frame

In [ ]:
J = puma.jacobe(puma.qn)

The Hessian is the derivative of the Jacobian and is a rank-3 tensor, a 3D array, useful in task-space dynamic problems

In [ ]:
H = puma.hessian0(puma.qn)
H.shape

For many robot problems we are interested in manipulability, that is, how well posed the robot is to make movement in any direction.  We can also think of this as distance away from a singularity where we lose a degree of freedom of motion.  Two common measures are due to Yoshikawa and Asada

In [ ]:
puma.manipulability(puma.qn)

In [ ]:
puma.manipulability(puma.qn, method="asada")

In [ ]:
puma.manipulability(puma.qn, axes="trans")

Motion control that optimizes manipulability needs the manipulability Jacobian, the rate of change of manipulability with respect to each joint variable

In [ ]:
puma.jacobm(puma.qn)

## E. Dynamics

The Toolbox has rich support for manipulator dynamics.

We can compute the inverse dynamics, the torques required to achieve a particular dynamic state $(q, \dot{q}, \ddot{q})$

In [ ]:
tau = puma.rne(puma.qn, np.zeros((6,)), np.zeros((6,)))

The inertia tensor is a symmetric matrix that depends on joint configuration

In [ ]:
J = puma.inertia(puma.qn)

and the velocity terms, Coriolis and centrigual coupling terms, is a function of joint configuration and velocity

In [ ]:
C = puma.coriolis(puma.qn, 0.1 * np.ones((6,)))

The final element is the gravity torque, the joint torques due to gravity weight forces acting on all the links

In [ ]:
g = puma.gravload(puma.qn)

The forward dynamics is the joint acceleration as a function of $(q, \dot{q}, \tau)$ and can be integrated to simulate the motion of the robot.

In [ ]:
qdd = puma.accel(puma.qn, np.zeros((6,)), tau)

# V. NEW CAPABILITY

## B. Collision checking

In [ ]:
from spatialgeometry import Cuboid


The Toolbox now has support for collision checking, using the bullet engine.

We create an instance of a Franka-Emika Panda robot and a big box, 1x1x1m 

In [ ]:
panda = models.URDF.Panda()
obstacle = Cuboid([1, 1, 1], base=SE3(1, 0, 0));

And then test if the robot, in configuration `qr`, collides with the box

In [ ]:
iscollision = panda.collided(panda.qr, obstacle) # boolean

which it does.  We can do more fine grained testing, does link[0] collide with the box?

In [ ]:
iscollision = panda.links[0].collided(obstacle)

No, it doesn't.  But what are the closest points on these objects?

In [ ]:

d, p1, p2 = panda.links[0].closest_point(obstacle)
print(d, p1, p2)

where `d` is the distance, 0.5m, and `p1` is a point on the robot link 0 and `p2` is a point on the box

## C. Interfaces

**The examples below will not work with CoLab.**  They require a local browser to display the robot.

To display the robot at a particular configuration

In [ ]:
panda.plot(panda.qr, block=False);

To animate the robot within the browser tab, we create a Swift instance, add the robot, and then, in a loop, update its configuration and `step` the simulation environment.

In [ ]:
from roboticstoolbox.backends.swift import Swift
qt = jtraj(panda.qz, panda.qr, 100)
backend = Swift()
backend.launch(realtime=True)  # create graphical world
backend.add(panda)             # add robot to the world
for q in qt.q:
    panda.q = q                # update the robot
    backend.step(dt=0.1)       # display/refresh the world
